In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import src
%matplotlib inline
df = pd.read_csv("../zippedData/tn.movie_budgets.csv.gz")
len(df)
df.head()
#importing everything, checking the length to account for changes during cleaning 

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


# Goal of Data Cleaning
The goal of this analysis is to examine movies at different budget levels by their ROI. To do se we added roi columns separated by worldwide and domestic, cleaned up the data and categorized each movies as either a small medium or large budget movie. 


In [3]:

#removes commas, dollar signs and converts all numerical data to float

df['worldwide_gross'] = src.string_to_float(df['worldwide_gross'])
df['production_budget'] = src.string_to_float(df['production_budget'])
df['domestic_gross'] = src.string_to_float(df['domestic_gross'])

#drops the first porno movie shown in theaters which made 175,500 times its budget
#Its both a massive outlier and presumably no longer possible in the industry microsoft is trying to break in to

df = df[df.movie != 'Deep Throat']

#getting rid of movies that have a 0 for worldwide gross AND domestic gross 
#some movies certainly make 0 dollars, but in this df its used as a placeholder
#an | is neccesary as a movie may only have made money worldwide or domestically

df = df[(df.worldwide_gross>0)|(df.domestic_gross>0)]

# taking out movies from before 1980 and checking the length to make sure this doesnt remove too much data

df['year'] = df['release_date'].apply(lambda x: int(x[-4:]))
df = df[df.year>1980]
len(df)

5135

In [4]:
df.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
year                 0
dtype: int64

In [5]:
#length looks good!
#no nan values either
#creating various profit and roi columns, by subtracting budget from gross and dividing by budget
df['profit'] = df['worldwide_gross'] - df['production_budget'] 
df['domestic_profit'] = df['domestic_gross'] - df['production_budget']
df['has foriegn'] = df['worldwide_gross'] > 0
df['roi'] = (df['profit']/ df['production_budget'])*100
df['domestic_roi'] = (df['domestic_profit']/ df['production_budget']) * 100
df['lost_money'] = df['profit']<0
#marking outliers in a new column using one of our src functions


df['outlier_budget'] = src.find_outliers(df, 'production_budget' )

    
#categorized each movie by the size of its budget as either 'large', 'medium' or 'small', our groupings
#are based on industry standards NOTE: these values are mostly arbitrary as different people may consider a
#$5 million not a small budget, theres no official definition however while not exact, they are within the parameter
#generally found in th industy. 

df['budget_category'] = df['production_budget'].apply(lambda x: src.categorize_production(x))

In [6]:
# exporting the cleaned data with updated columns to a csv for use in other notebooks
df.to_csv(r'/Users/aidancoco/desktop/Flatiron/Projects/movie/movie-data-eda/aidan_work\budget_roi_data_clean.csv', index = False)

In [7]:
df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,profit,domestic_profit,has foriegn,roi,domestic_roi,lost_money,outlier_budget,budget_category
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2009,2.351345e+09,335507625.0,True,553.257713,78.942971,False,True,large
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,6.350639e+08,-169536125.0,True,154.667286,-41.289850,False,True,large
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,-2.002376e+08,-307237650.0,True,-57.210757,-87.782186,True,True,large
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,1.072414e+09,128405868.0,True,324.384139,38.840250,False,True,large
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,9.997217e+08,303181382.0,True,315.369636,95.640815,False,True,large
